In [1]:
import pandas as pd

from datetime import datetime,timedelta

In [2]:
df = pd.read_csv('big mac.csv')

In [3]:
df

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
0,2000-04-01,ARG,ARS,Argentina,2.50,1.0000,2.500000,-0.00398,0.05007,-0.16722,-0.09864,1.09091,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-04-01,AUS,AUD,Australia,2.59,1.6800,1.541667,-0.38579,-0.35246,-0.48645,-0.44416,0.28939,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-04-01,BRA,BRL,Brazil,2.95,1.7900,1.648045,-0.34341,-0.30778,-0.45102,-0.40581,0.37836,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,CAN,CAD,Canada,2.85,1.4700,1.938776,-0.22758,-0.18566,-0.35417,-0.30099,0.62152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-04-01,CHE,CHF,Switzerland,5.90,1.7000,3.470588,0.38270,0.45774,0.15609,0.25130,1.90267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2020-07-01,UKR,UAH,Ukraine,59.00,27.1300,2.174714,-0.61914,-0.54562,-0.49157,-0.40181,-0.29813,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1382,2020-07-01,URY,UYU,Uruguay,189.00,43.6750,4.327418,-0.24213,-0.09584,0.01171,0.19032,0.39664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1383,2020-07-01,USA,USD,United States,5.71,1.0000,5.710000,0.00000,0.19303,0.33494,0.57062,0.84286,62868.917,5.174502,0.000,-0.019,0.122,0.279,0.070
1384,2020-07-01,VNM,VND,Vietnam,66000.00,23180.0000,2.847282,-0.50135,-0.40510,-0.33433,-0.21681,-0.08106,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_frames = {}
countries = set(df["name"].tolist())
print(len(countries))
to_drop = ["USD_raw", "EUR_raw", "GBP_raw", "JPY_raw", "CNY_raw", "GDP_dollar", "adj_price", "USD_adjusted", "EUR_adjusted", "GBP_adjusted", "JPY_adjusted", "CNY_adjusted", ]

57


In [5]:
def change_date_to_datetime(str):
    datetime_object = datetime.strptime(str, '%Y-%m-%d').date()
    return datetime_object


In [6]:
#change str to datetime object

df["date"] = df["date"].apply(change_date_to_datetime)
columns_tochange = ["local_price", "dollar_ex", "dollar_price"]

In [7]:
df = df.drop(to_drop, axis=1)

In [8]:
for country in countries:
    data_frames[country] = df[df["name"] == country]

Seperating Data frames by Country

In [9]:
year1999 = datetime(1999, 12, 31).date()
year2021 = datetime(2021, 1, 1).date()
x = data_frames["Canada"]["date"].iloc[0]

In [10]:
def nearest(items, pivot):
    return min(items, key=lambda z: abs(z - pivot))

In [11]:
years = []
for i in range(2000, 2021):
    years.append(datetime(i, 1, 1).date())
for country in countries:
    # Enure all data is is in range 2000 - 2020
    frame = data_frames[country]
    data_frames[country] = frame[((frame["date"] > year1999) &  (year2021 > frame["date"]))]
    for year in years:
        x = frame[((frame["date"] >= year) &  ((year+pd.DateOffset(years=1)) > frame["date"]))]
        if len(x) > 1:
            y = x["date"].tolist()
            midlest = nearest(y, (year+timedelta(days=150)))
            to_delete = x[((x["date"] > midlest) |  (midlest > x["date"]))]
            data_frames[country] = pd.concat([data_frames[country], to_delete])
    data_frames[country] = data_frames[country].drop_duplicates(keep=False)
        

In [12]:
final_list = []
for country in countries.copy():
    if len(data_frames[country]) < 21:
        countries.remove(country)
        del data_frames[country]
    else:
        final_list.append(data_frames[country])
        
final_df = pd.concat(final_list)

In [13]:
final_df = final_df.reset_index(drop=True)
final_df

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
0,2000-04-01,BRA,BRL,Brazil,2.95,1.790,1.648045
1,2001-04-01,BRA,BRL,Brazil,3.60,2.190,1.643836
2,2002-04-01,BRA,BRL,Brazil,3.60,2.340,1.538462
3,2003-04-01,BRA,BRL,Brazil,4.55,3.070,1.482085
4,2004-05-01,BRA,BRL,Brazil,5.40,3.180,1.698113
...,...,...,...,...,...,...,...
562,2016-07-01,JPN,JPY,Japan,370.00,106.725,3.466854
563,2017-07-01,JPN,JPY,Japan,380.00,113.060,3.361047
564,2018-07-01,JPN,JPY,Japan,390.00,111.250,3.505618
565,2019-07-09,JPN,JPY,Japan,390.00,108.765,3.585712


In [14]:
final_df.to_csv("cleaned_bigmac.csv", index=False)

In [15]:
gdp_data = pd.read_csv("gdp_data.csv")

In [40]:
gdp_list = []

for year in years:
    year_set = gdp_data[["Country Code", str(year.year)]]
    gdp_list.append(year_set)


In [41]:
gdp_data2 = pd.concat(gdp_list)

In [58]:
def get_year(x):
    return x.year

In [65]:
final_df["GDP"] = (gdp_data2[gdp_data2["Country Code"] == str(final_df["iso_a3"])][str(final_df["date"].apply(get_year))]).dropna().iloc[0]

KeyError: '0      2000\n1      2001\n2      2002\n3      2003\n4      2004\n       ... \n562    2016\n563    2017\n564    2018\n565    2019\n566    2020\nName: date, Length: 567, dtype: int64'

In [72]:
def get_gdp(row):
    # Convert date to year using get_year function and apply filtering
    filtered = gdp_data2[gdp_data2["Country Code"] == str(row["iso_a3"])][str(row["date"].year)].dropna()
    
    # Check if the filtered DataFrame is not empty
    if not filtered.empty:
        return filtered.iloc[0]  # Access the first matching GDP value
    else:
        return pd.NA  # Return None or a default value if no match is found

# Apply the function to the DataFrame row by row
final_df["GDP"] = final_df.apply(get_gdp, axis=1)


In [73]:
final_df

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,GDP
0,2000-04-01,BRA,BRL,Brazil,2.95,1.790,1.648045,3726.81
1,2001-04-01,BRA,BRL,Brazil,3.60,2.190,1.643836,3142.24
2,2002-04-01,BRA,BRL,Brazil,3.60,2.340,1.538462,2824.72
3,2003-04-01,BRA,BRL,Brazil,4.55,3.070,1.482085,3056.65
4,2004-05-01,BRA,BRL,Brazil,5.40,3.180,1.698113,3623.22
...,...,...,...,...,...,...,...,...
562,2016-07-01,JPN,JPY,Japan,370.00,106.725,3.466854,39375.5
563,2017-07-01,JPN,JPY,Japan,380.00,113.060,3.361047,38834.1
564,2018-07-01,JPN,JPY,Japan,390.00,111.250,3.505618,39751.1
565,2019-07-09,JPN,JPY,Japan,390.00,108.765,3.585712,40416


In [74]:
final_df.to_csv("cleaned_bigmac.csv", index=False)
